# Access HATS catalogs from IPAC's LSDB-service 

In [1]:
# !pip install requests astropy

In [2]:
import requests
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord
import pyvo

In [3]:
SERVICE_URL = "https://troyraen.irsakudev.ipac.caltech.edu" # the LSDB service deployed at IPAC (need to be on IPAC network)
# SERVICE_URL = "http://epyc.astro.washington.edu:43212" # will work once Derek deploys the server uncoupled from Firefly

SERVICE_API_ENDPOINT = "cgi-bin/Gator" # would be /TAP or something if a VO interface is developed

In [4]:
requests.get(f"{SERVICE_URL}/{SERVICE_API_ENDPOINT}").text

'lsdb-server is ready.'

## Get catalogs from a project

In [5]:
PROJECT_NAME = "WISE"

In [6]:
response = requests.get(f"{SERVICE_URL}/{SERVICE_API_ENDPOINT}/nph-scan",
                        params={"projshort": PROJECT_NAME})
response.raise_for_status()

catalogs = Table.read(response.text, format='ipac')
catalogs

hats_dataset,cntr,description,subtitle,ack,infourl,server,database,catname,ddname,ddlink,xpffile,cols,nrows,ext,key,pos,simulated,rel,coneradius,uploadradius,projectshort,missionid,groupid,access_flag,alg,algv,ex,where1,where2,mojoin,modataset,toperiod,periodresult,sptlevel,sptindex,visibility,dbms
str38,str1,str49,str14,str7,str83,str12,str12,str12,str7,str71,str7,int64,str14,str7,str7,str1,str1,str7,int64,int64,str4,str7,str7,str7,str7,str7,str1,float64,str1,str7,str7,str7,str7,str7,str7,str7,str7
https://data.lsdb.io/hats/wise/allwise,6,AllWISE Source Catalog,AllWISE,NEEDED?,https://wise2.ipac.caltech.edu/docs/release/allwise/,data.lsdb.io,wise,allwise,NEEDED?,https://wise2.ipac.caltech.edu/docs/release/allwise/expsup/sec2_1a.html,NEEDED?,302,"747,634,026",NEEDED?,NEEDED?,y,n,NEEDED?,3600,1200,WISE,NEEDED?,NEEDED?,NEEDED?,NEEDED?,NEEDED?,--,--,--,NEEDED?,NEEDED?,NEEDED?,NEEDED?,NEEDED?,NEEDED?,NEEDED?,NEEDED?
https://data.lsdb.io/hats/wise/neowise,7,NEOWISE Year 8 Single Exposure (L1b) Source Table,NEOWISE Year 8,NEEDED?,https://wise2.ipac.caltech.edu/docs/release/neowise/neowise_2022_release_intro.html,data.lsdb.io,neowiser_yr8,neowiser_yr8,NEEDED?,https://wise2.ipac.caltech.edu/docs/release/neowise/expsup/sec2_1a.html,NEEDED?,146,"18,668,416,994",NEEDED?,NEEDED?,y,n,NEEDED?,3600,180,WISE,NEEDED?,NEEDED?,NEEDED?,NEEDED?,NEEDED?,--,--,--,NEEDED?,NEEDED?,NEEDED?,NEEDED?,NEEDED?,NEEDED?,NEEDED?,NEEDED?


In [7]:
# Do same using pyvo/astroquery API had it been a VO service

## Get a catalog table's schema (columns and their metadata)

In [8]:
CATALOG_NAME = str(catalogs['catname'][0])  # pick the first catalog
CATALOG_NAME

'allwise'

In [9]:
response = requests.get(f"{SERVICE_URL}/{SERVICE_API_ENDPOINT}/nph-dd",
                        params={"catalog": CATALOG_NAME})
response.raise_for_status()

schema = Table.read(response.text, format='ipac')
schema

cntr,name,dbtype,description,format,groupid,indx,irsadef,nulls,sel,tableflg,units
int64,str13,str14,str1,str14,int64,str1,str1,str1,str1,int64,str1
0,_healpix_29,NUMBER(19),--,NUMBER(19),0,n,n,y,n,2,--
1,designation,NVARCHAR2(255),--,NVARCHAR2(255),1,n,n,y,n,2,--
2,ra,FLOAT(53),--,FLOAT(53),2,n,n,y,y,2,--
3,dec,FLOAT(53),--,FLOAT(53),3,n,n,y,y,2,--
4,sigra,FLOAT(53),--,FLOAT(53),4,n,n,y,n,2,--
5,sigdec,FLOAT(53),--,FLOAT(53),5,n,n,y,n,2,--
6,sigradec,FLOAT(53),--,FLOAT(53),6,n,n,y,n,2,--
7,glon,FLOAT(53),--,FLOAT(53),7,n,n,y,n,2,--
8,glat,FLOAT(53),--,FLOAT(53),8,n,n,y,n,2,--


In [10]:
# Do same using pyvo/astroquery API had it been a VO service

## Get data from a catalog

### Spatial filter

In [11]:
SPATIAL_REGION = "cone"

# change as needed
TARGET = SkyCoord("19h17m32s 11d58m02s")
RADIUS = 50 * u.arcsec

### Columns filter

In [12]:
COLUMNS = ["ra", "dec", "cntr", "w1mpro", "w2mpro", "w1snr", "w2snr"]

### Rows filter

In [13]:
ROW_FILTER = "w1snr>10 & w2snr>10"

### Query the catalog

In [14]:
response = requests.get(f"{SERVICE_URL}/{SERVICE_API_ENDPOINT}/nph-query",
                        params={"catalog": CATALOG_NAME,
                                "selcols": ",".join(COLUMNS),
                                "spatial": SPATIAL_REGION,
                                "objstr": f"{TARGET.to_string("hmsdms")} equ j2000",
                                "radius": RADIUS.value,
                                "radunits": RADIUS.unit.name,
                                "constraints": ROW_FILTER,
                                })
response.raise_for_status()

data = Table.read(response.text, format='ipac')
data

ra,dec,cntr,w1mpro,w1snr,w2mpro,w2snr
float64,float64,int64,float64,float64,float64,float64
289.38283,11.9549882,2889112101351020632,10.225,35.3,10.126,25.2
289.3874535,11.9562289,2889112101351020195,8.266,47.4,8.302,49.6
289.374699,11.9608758,2889112101351028286,11.1,28.3,10.949,20.6
289.3766927,11.9619711,2889112101351028307,11.151,29.9,10.979,21.2
289.3952792,11.9704945,2889112101351029534,12.353,22.7,11.652,23.7
289.3710908,11.9740983,2889112101351028612,11.755,18.6,11.742,22.4
289.3857219,11.9795382,2889112101351028215,10.72,35.4,10.823,42.5
289.3740138,11.9747719,2889112101351028631,11.667,20.1,11.447,26.5
289.3787859,11.9792457,2889112101351028164,11.217,19.4,10.934,18.9


In [15]:
# Do same using pyvo/astroquery API had it been a VO service